## Requirements
Install plotly: ```pip install plotly```

In [12]:
import requests
import time
from datetime import date,datetime,timezone
import pandas as pd
import plotly.graph_objects as go

# Settings
finn_key = "btv5stv48v6q7nvmtsjg"
base_url = "https://finnhub.io/api/v1/stock/candle"

In [13]:
# This function pulls and plots daily stock prices for a company ('symbol') over a given date range and overlays Bollinger Band chart indicators
def bollinger(symbol, start_date, end_date):
    # Format API call with passed in parameters
    params = {'symbol': symbol,
              'resolution': 'D',
              'from': start_date,
              'to': end_date,
              'token': finn_key}
    response = requests.get(base_url,params=params).json()
    
    # Convert API response to dataframe
    df = pd.DataFrame(response)
    
    # Create date column that is human readable
    df['date'] = [datetime.utcfromtimestamp(d) for d in df['t']] # utcfromtimestamp for accurate date
    # Typical price is the average of the high, low and close
    df['tp'] = (df.h + df.l + df.c) / 3.0
    
    df.set_index(pd.DatetimeIndex(df['date']), inplace=True) # needs datetime as index for rolling offset
    
    # 20 day moving average of the typical price
    df['ma20']= df.tp.rolling('20D').mean() # 20 day offset
    # 20 day standard deviation of the typical price
    df['std20']= df.tp.rolling('20D').std()
    
    # Define upper and lower Bollinger bands as 2 deviations above and below the moving average
    df['upper'] = df['ma20'] + 2*df['std20']
    df['lower'] = df['ma20'] - 2*df['std20']
#     print(df)
    
    # Create a plot using plotly with upper and lower Bollinger bands,
    # a plot for the moving average, 
    # a candlestick plot for the stock we're interested in
    fig = go.Figure(data = [
        go.Scatter(x=df['date'],y=df['lower'],line_color='gray',name='Lower Bollinger Band'),
        go.Scatter(x=df['date'],y=df['upper'],line_color='gray',name='Upper Bollinger Band',fill='tonexty'),
        go.Scatter(x=df['date'],y=df['ma20'],line=dict(color='gray',dash='dash'),name='20 Day Moving Average'),
        go.Candlestick(
            x=df['date'],
            open=df['o'],
            high=df['h'],
            low=df['l'],
            close=df['c'],
            name=symbol
        )
    ])
    fig.update_layout(
    title="{} with 20-day, two-standard-deviation Bollinger Bands".format(symbol),
    xaxis_title="Date",
    yaxis_title="Price ($)",
#     legend_title="Legend Title",
    )
    # Allow for adjusting the y-axis
    fig.update_yaxes(fixedrange = False)
    return fig

In [15]:
stocks = ['AMZN','AAPL','MSFT','TSLA']

end_date = 1603238400 # Oct 21, 2020
# start_date = end_date - 2 years
start_date = end_date-2*365*24*3600-86400 # Oct 21, 2018 need to subtract extra day

for stock in stocks:
    fig = bollinger(stock, start_date, end_date)
    fig.show()

## Analysis
Out of the four companies we analyzed, Amazon has the most potential for a short-term upwards price movement based on its Bollinger Bands.
The bands are wide apart, meaning the stock price is volatile and is likely due for a big swing in either direction.
Amazon’s stock price is trending towards the lower Bollinger Band at `$3,051` which indicates that the stock is oversold, cheaper than it should be. We predict that the stock price will move upwards to at least the 20 day moving average of `$3,259`.

After Amazon, Microsoft may be the most volatile with its Bollinger Bands widening.

The Bollinger Bands suggest that APPL, MSFT, and TSLA will not see much short-term price movement. All three stock prices are closely centered around their 20 day moving averages.
